## Agent with Memory:

### <u>Memory Saver</u>:

* LangGraph can use a checkpoint to automatically save the graph state after each step.

* This built-in persistent layer gives us memory, allowing LangGraph to pick up from the last state update.

* One of the easiest checkpointers to use is the `MemorySaver`, an in-memory key-value store for Graph state.

* All we need to do is simply compile the grpah with a checkpointer, add thread to config parameter, and out graph has memory!

In [1]:
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "llama3.2")

In [6]:
from datetime import datetime

def date():
    """Returns the present date, month and year as datetime object
    args:none
    return: date, month, year as datetime object
    """
    return datetime.now().date()

def time():
    """Return the time in Hrs : Min : Sec : Milli_seconds, as datetime object
    args:none
    return: the current time in hrs : min : sec : milli_seconds
    """
    return datetime.now().time()

In [13]:
tools = [date,time]

In [14]:
llm_with_tools = llm.bind_tools(tools)

In [15]:
def llm_calling_tools(state:State):
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

In [16]:
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)

builder.add_node('llm_tooling',llm_calling_tools)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START,"llm_tooling")
builder.add_conditional_edges("llm_tooling",tools_condition)
builder.add_edge("tools","llm_tooling")
builder.add_edge("llm_tooling",end_key=END)

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
builder_graph = builder.compile(memory)

In [21]:
config = {'configurable':{'thread_id':'2'}}

In [22]:
from pprint import pprint
response = builder_graph.invoke({'messages':"I want to know the current time"},config=config)

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time
================================== Ai Message ==================================
Tool Calls:
  time (61b8f07e-042e-4fd0-add0-36434162c09f)
 Call ID: 61b8f07e-042e-4fd0-add0-36434162c09f
  Args:
================================= Tool Message =================================
Name: time

19:19:21.187231
================================== Ai Message ==================================

The current time is 7:19 PM.


In [23]:
response = builder_graph.invoke({'messages':"Now, what will be the time if 2 hrs will pass"},config=config)

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time
================================== Ai Message ==================================
Tool Calls:
  time (61b8f07e-042e-4fd0-add0-36434162c09f)
 Call ID: 61b8f07e-042e-4fd0-add0-36434162c09f
  Args:
================================= Tool Message =================================
Name: time

19:19:21.187231
================================== Ai Message ==================================

The current time is 7:19 PM.
================================ Human Message =================================

Now, what will be the time if 2 hrs will pass
================================== Ai Message ==================================
Tool Calls:
  time (76636baa-598c-49e2-b741-a0199618267b)
 Call ID: 76636baa-598c-49e2-b741-a0199618267b
  Args:
    t: 7200000
================================= Tool Message =================================
Name: time

19:19:48.515220
==========================

In [24]:
response = builder_graph.invoke({'messages':"what is the date today?"},config=config)

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time
================================== Ai Message ==================================
Tool Calls:
  time (61b8f07e-042e-4fd0-add0-36434162c09f)
 Call ID: 61b8f07e-042e-4fd0-add0-36434162c09f
  Args:
================================= Tool Message =================================
Name: time

19:19:21.187231
================================== Ai Message ==================================

The current time is 7:19 PM.
================================ Human Message =================================

Now, what will be the time if 2 hrs will pass
================================== Ai Message ==================================
Tool Calls:
  time (76636baa-598c-49e2-b741-a0199618267b)
 Call ID: 76636baa-598c-49e2-b741-a0199618267b
  Args:
    t: 7200000
================================= Tool Message =================================
Name: time

19:19:48.515220
==========================

### Lets add another tool to calculate the day:

In [48]:
def day():
    """Tells us which day is today. Returns the day in string format
    args:none
    output: return which day is today in string format
    """
    day_number = datetime.date().isoweekday()

    dn = {
        1 : 'Monday',
        2 : 'Tuesday',
        3 : 'Wednesday',
        4 : 'Thursday',
        5 : 'Friday',
        6 : 'Saturday',
        7 : 'Sunday'
    }    
    return day_number[dn]

In [52]:
new_tools = [date, time, day]

llm_with_tools1 = llm.bind_tools(new_tools)

In [53]:
def llm_calling_tools1(state:State):
    return {'messages': [llm_with_tools1.invoke(state['messages'])]}

builder1 = StateGraph(State)

builder1.add_node('llm_tooling1',llm_calling_tools1)
builder1.add_node("tools", ToolNode(new_tools))

builder1.add_edge(START,"llm_tooling1")
builder1.add_conditional_edges("llm_tooling1",tools_condition)
builder1.add_edge("tools","llm_tooling1")
builder1.add_edge("llm_tooling1",end_key=END)

builder_graph1 = builder1.compile(memory)

In [54]:
response = builder_graph1.invoke({'messages':"what will be the date on wednesday?"},config=config)

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time
================================== Ai Message ==================================
Tool Calls:
  time (61b8f07e-042e-4fd0-add0-36434162c09f)
 Call ID: 61b8f07e-042e-4fd0-add0-36434162c09f
  Args:
================================= Tool Message =================================
Name: time

19:19:21.187231
================================== Ai Message ==================================

The current time is 7:19 PM.
================================ Human Message =================================

Now, what will be the time if 2 hrs will pass
================================== Ai Message ==================================
Tool Calls:
  time (76636baa-598c-49e2-b741-a0199618267b)
 Call ID: 76636baa-598c-49e2-b741-a0199618267b
  Args:
    t: 7200000
================================= Tool Message =================================
Name: time

19:19:48.515220
==========================

In [55]:
response = builder_graph1.invoke({'messages':"which day will be on 2nd of october 2025?"},config=config)

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to know the current time
================================== Ai Message ==================================
Tool Calls:
  time (61b8f07e-042e-4fd0-add0-36434162c09f)
 Call ID: 61b8f07e-042e-4fd0-add0-36434162c09f
  Args:
================================= Tool Message =================================
Name: time

19:19:21.187231
================================== Ai Message ==================================

The current time is 7:19 PM.
================================ Human Message =================================

Now, what will be the time if 2 hrs will pass
================================== Ai Message ==================================
Tool Calls:
  time (76636baa-598c-49e2-b741-a0199618267b)
 Call ID: 76636baa-598c-49e2-b741-a0199618267b
  Args:
    t: 7200000
================================= Tool Message =================================
Name: time

19:19:48.515220
==========================